<div style="background: linear-gradient(to right,rgb(225, 231, 134),rgb(176, 238, 148),rgb(150, 232, 238)); padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 10px 20px rgba(0,0,0,0.19), 0 6px 6px rgba(0,0,0,0.23);">
    <span style="font-family: 'Montserrat', sans-serif; font-weight: 800; font-size: 2.5em; color: white; text-shadow: 2px 2px 4px #000;">✨ IMPORTS ✨</span>
</div>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



SPLITTING DATA

In [ ]:
# load data, labels, patients from pkl files
import pickle
import os

def load_data(data_path, labels_path, patients_path):
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
    with open(labels_path, 'rb') as f:
        labels = pickle.load(f)
    with open(patients_path, 'rb') as f:
        patients = pickle.load(f)
    return data, labels, patients

data,labels, patients = load_data('data.pkl', 'labels.pkl', 'patients.pkl')


In [ ]:

import numpy as np

def separate_data_by_class(labels,data, patients, class_no):
    class_indices = np.where(labels == int(class_no))[0]
    data_class = data[class_indices]
    labels_class = labels[class_indices]
    patients_class = patients[class_indices]
    return class_indices, data_class, labels_class, patients_class


def create_patient_folds(patients, n_splits=5):
    all_unique_patients = np.unique(patients)
    print("No. of unique patients is:", len(all_unique_patients))
    np.random.shuffle(all_unique_patients)
    patient_splits = np.array_split(all_unique_patients, n_splits)

    patient_fold_mapping = {}
    for fold, patient_group in enumerate(patient_splits):
        for patient in patient_group:
            patient_fold_mapping[patient] = fold

    return patient_fold_mapping

def split_data_by_patients_across_classes(eeg_data, labels, patients, patient_fold_mapping, n_splits=5):
    eeg_data_splits = [[] for _ in range(n_splits)]
    labels_splits = [[] for _ in range(n_splits)]
    patients_splits = [[] for _ in range(n_splits)]
    for i in range(len(patients)):
        patient_id = patients[i]
        fold = patient_fold_mapping[patient_id]  # Get the fold number from the mapping
        eeg_data_splits[fold].append(eeg_data[i])
        labels_splits[fold].append(labels[i])
        patients_splits[fold].append(patients[i])
        
    eeg_data_splits = [np.array(split) for split in eeg_data_splits]
    labels_splits = [np.array(split) for split in labels_splits]
    patients_splits = [np.array(split) for split in patients_splits]

    return eeg_data_splits, labels_splits, patients_splits


def shuffle_fold(eeg_fold, labels_fold, patients_fold):
    indices = np.arange(len(eeg_fold))  
    np.random.shuffle(indices)          
    eeg_fold_shuffled = eeg_fold[indices]
    labels_fold_shuffled = labels_fold[indices]
    patients_fold_shuffled = patients_fold[indices]

    return eeg_fold_shuffled, labels_fold_shuffled, patients_fold_shuffled


c0_indices, data_c0, labels_c0, patients_c0 = separate_data_by_class(labels, data, patients, 0)

c1_indices, data_c1, labels_c1, patients_c1 = separate_data_by_class(labels, data, patients, 1)

c2_indices, data_c2, labels_c2, patients_c2 = separate_data_by_class(labels, data, patients, 2)


print(f"Class 0: {data_c0.shape}, {labels_c0.shape}, {len(np.unique(patients_c0))}")
print(f"Class 1: {data_c1.shape}, {labels_c1.shape}, {len(np.unique(patients_c1))}")
print(f"Class 2: {data_c2.shape}, {labels_c2.shape}, {len(np.unique(patients_c2))}")


patients_set_c0 = set(patients_c0)
patients_set_c1 = set(patients_c1)
patients_set_c2 = set(patients_c2)

common_patients_all = patients_set_c0.intersection(patients_set_c1, patients_set_c2)
common_patients_all = list(common_patients_all)
print('No. of common patients between all classes: ', len(common_patients_all))

def extract_common_data(patients, data, labels, common_patient):
    common_indices = [i for i, patient in enumerate(patients) if patient in common_patient]
    common_data = data[common_indices]
    common_labels = labels[common_indices]
    common_patients = patients[common_indices]
    return common_data, common_labels, common_patients

common_data_c0, common_labels_c0, common_patients_c0 = extract_common_data(patients_c0, data_c0, labels_c0, common_patients_all)
common_data_c1, common_labels_c1, common_patients_c1 = extract_common_data(patients_c1, data_c1, labels_c1, common_patients_all)
common_data_c2, common_labels_c2, common_patients_c2 = extract_common_data(patients_c2, data_c2, labels_c2, common_patients_all)

print(np.unique(common_patients_c0).shape)

common_allpatient_fold_mapping = create_patient_folds(common_patients_all, n_splits=5)
eeg_012_c0, labels_012_c0, patients_012_c0 = split_data_by_patients_across_classes(
    common_data_c0, common_labels_c0, common_patients_c0, common_allpatient_fold_mapping, n_splits=5)

eeg_012_c1, labels_012_c1, patients_012_c1 = split_data_by_patients_across_classes(
    common_data_c1, common_labels_c1, common_patients_c1, common_allpatient_fold_mapping, n_splits=5)

eeg_012_c2, labels_012_c2, patients_012_c2 = split_data_by_patients_across_classes(
    common_data_c2, common_labels_c2, common_patients_c2, common_allpatient_fold_mapping, n_splits=5)

for i in range(5):
    print(f"Fold {i+1}: EEG shape: {np.concatenate([eeg_012_c0[i], eeg_012_c1[i], eeg_012_c2[i]]).shape}, "
          f"Labels shape: {np.concatenate([labels_012_c0[i], labels_012_c1[i], labels_012_c2[i]]).shape}, "
          f"Patients shape: {np.concatenate([patients_012_c0[i], patients_012_c1[i], patients_012_c2[i]]).shape}")


for i in range(5):
    print(f"Fold {i+1}: EEG shape 0: {eeg_012_c0[i].shape}, "
          f"Labels shape 0: {labels_012_c0[i].shape}, "
          f"Patients shape 0: {patients_012_c0[i].shape}")
print("__________________________________\n")

for i in range(5):
    print(f"Fold {i+1}: EEG shape 1: {eeg_012_c1[i].shape}, "
      f"Labels shape 1: {labels_012_c1[i].shape}, "
      f"Patients shape 1: {patients_012_c1[i].shape}")

print("__________________________________\n")
for i in range(5):
    print(f"Fold {i+1}: EEG shape 2: {eeg_012_c2[i].shape}, "
      f"Labels shape 2: {labels_012_c2[i].shape}, "
      f"Patients shape 2: {patients_012_c2[i].shape}")
    
for i in range(5):
    print(f"No. of patients in class 2 is {len(np.unique(patients_012_c2[i]))}")
for i in range(5):
    print(f"No. of patients in class 1 is {len(np.unique(patients_012_c1[i]))}")
for i in range(5):
    print(f"No. of patients in class 0 is {len(np.unique(patients_012_c0[i]))}")
    
    # Find exclusive patients for each class
exclusive_patients_c0 = list(patients_set_c0 - set(common_patients_all))
exclusive_patients_c1 = list(patients_set_c1 - set(common_patients_all))
exclusive_patients_c2 = list(patients_set_c2 - set(common_patients_all))

print(len(exclusive_patients_c0))
print(len(exclusive_patients_c1))
print(len(exclusive_patients_c2))

exclusive_c2 = set(exclusive_patients_c2)
exclusive_c0 = set(exclusive_patients_c0)
common_c2_c0 = exclusive_c0.intersection(exclusive_c2)
common_patient_c2_c0 = list(common_c2_c0)

print(len(common_patient_c2_c0))



common_data_c02, common_labels_c02, common_patients_c02 = extract_common_data(patients_c0, data_c0, labels_c0, common_patient_c2_c0)
common_data_c22, common_labels_c22, common_patients_c22 = extract_common_data(patients_c2, data_c2, labels_c2, common_patient_c2_c0)

print(np.unique(common_patients_c02).shape)
print(np.unique(common_patients_c22).shape)

c20_mapping = create_patient_folds(common_patient_c2_c0, n_splits=5)

common_eeg_splits_c02, common_labels_splits_c02, common_patients_splits_c02 = split_data_by_patients_across_classes(
    common_data_c02, common_labels_c02, common_patients_c02, c20_mapping, n_splits=5)

common_eeg_splits_c22, common_labels_splits_c22, common_patients_splits_c22 = split_data_by_patients_across_classes(
    common_data_c22, common_labels_c22, common_patients_c22, c20_mapping, n_splits=5)


# Check the shape of each fold for verification
for i in range(5):
    print(f"Fold {i+1}: EEG shape: {np.concatenate([common_eeg_splits_c02[i], common_eeg_splits_c22[i]]).shape}, "
          f"Labels shape: {np.concatenate([common_labels_splits_c02[i], common_labels_splits_c22[i]]).shape}, "
          f"Patients shape: {np.concatenate([common_patients_splits_c02[i], common_patients_splits_c22[i]]).shape}")

for i in range(5):
    print(f"Fold {i+1}: EEG shape 0: {common_eeg_splits_c02[i].shape}, "
          f"Labels shape 0: {common_labels_splits_c02[i].shape}, "
          f"Patients shape 0: {common_patients_splits_c02[i].shape}")
print("__________________________________\n")

for i in range(5):
    print(f"Fold {i+1}: EEG shape 1: {common_eeg_splits_c22[i].shape}, "
      f"Labels shape 1: {common_labels_splits_c22[i].shape}, "
      f"Patients shape 1: {common_patients_splits_c22[i].shape}")

print("__________________________________\n")

for i in range(5):
    print(f"No. of patients in class 1 is {len(np.unique(common_patients_splits_c02[i]))}")
for i in range(5):
    print(f"No. of patients in class 2 is {len(np.unique(common_patients_splits_c22[i]))}")


# Find exclusive patients for each class
exclusive_patients_c0_c2 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0))
exclusive_patients_c1_c2 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0))
exclusive_patients_c2_c2 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0))

print(len(exclusive_patients_c0_c2))
print(len(exclusive_patients_c1_c2))
print(len(exclusive_patients_c2_c2))

exclusive_c2 = set(exclusive_patients_c2_c2)
exclusive_c1 = set(exclusive_patients_c1_c2)
common_c2_c1 = exclusive_c1.intersection(exclusive_c2)
common_patient_c2_c1 = list(common_c2_c1)

print(len(common_patient_c2_c1))



common_data_c12, common_labels_c12, common_patients_c12 = extract_common_data(patients_c1, data_c1, labels_c1, common_patient_c2_c1)
common_data_1_c22, common_labels_1_c22, common_patients_1_c22 = extract_common_data(patients_c2, data_c2, labels_c2, common_patient_c2_c1)

print(np.unique(common_patients_c12).shape)
print(np.unique(common_patients_1_c22).shape)

c21_mapping = create_patient_folds(common_patient_c2_c1, n_splits=5)

common_eeg_splits_c12, common_labels_splits_c12, common_patients_splits_c12 = split_data_by_patients_across_classes(
    common_data_c12, common_labels_c12, common_patients_c12, c21_mapping, n_splits=5)

common_eeg_splits_1_c22, common_labels_splits_1_c22, common_patients_splits_1_c22 = split_data_by_patients_across_classes(
    common_data_1_c22, common_labels_1_c22, common_patients_1_c22, c21_mapping, n_splits=5)


# Check the shape of each fold for verification
for i in range(5):
    print(f"Fold {i+1}: EEG shape: {np.concatenate([common_eeg_splits_c12[i], common_eeg_splits_1_c22[i]]).shape}, "
          f"Labels shape: {np.concatenate([common_labels_splits_c12[i], common_labels_splits_1_c22[i]]).shape}, "
          f"Patients shape: {np.concatenate([common_patients_splits_c12[i], common_patients_splits_1_c22[i]]).shape}")


for i in range(5):
    print(f"No. of patients in class 1 is {len(np.unique(common_patients_splits_c12[i]))}")
for i in range(5):
    print(f"No. of patients in class 2 is {len(np.unique(common_patients_splits_1_c22[i]))}")
    

exclusive_patients_c0_c2_c1 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0) - set(common_c2_c1))
exclusive_patients_c1_c2_c1 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1))
exclusive_patients_c2_c2_c1 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0)- set(common_c2_c1))

print(len(exclusive_patients_c0_c2_c1))
print(len(exclusive_patients_c1_c2_c1))
print(len(exclusive_patients_c2_c2_c1))

only_c2 = set(exclusive_patients_c2_c2_c1)
only_c2= list(only_c2)
data_only_c2, labels_only_c2,patients_only_c2 = extract_common_data(patients_c2, data_c2, labels_c2, only_c2)
print(np.unique(patients_only_c2).shape)

c2_mapping = create_patient_folds(only_c2, n_splits=5)

eeg_splits_c2, labels_splits_c2, patients_splits_c2 = split_data_by_patients_across_classes(
    data_only_c2, labels_c2, patients_only_c2, c2_mapping, n_splits=5)

for i in range(5):
    print(f"No. of patients in class 2 is {len(np.unique(patients_splits_c2[i]))}")
    
    # Find exclusive patients for each class
exclusive_patients_all_c02 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0) - set(common_c2_c1)- set(only_c2))
exclusive_patients_all_c12 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2))
exclusive_patients_all_c22 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2))

print(len(exclusive_patients_all_c02))
print(len(exclusive_patients_all_c12))
print(len(exclusive_patients_all_c22))


exc_c1_c11 = set(exclusive_patients_all_c12)
exc_c1_c01 = set(exclusive_patients_all_c02)
common_patient_c1_c0 = exc_c1_c01.intersection(exc_c1_c11)
common_patient_c1_c0 = list(common_patient_c1_c0)

print(len(common_patient_c1_c0))

common_data_c1_c01, common_labels_c1_c01, common_patients_c1_c01 = extract_common_data(patients_c0, data_c0, labels_c0, common_patient_c1_c0)
common_data_c1_c11, common_labels_c1_c11, common_patients_c1_c11 = extract_common_data(patients_c1, data_c1, labels_c1, common_patient_c1_c0)

print(np.unique(common_patients_c1_c01).shape)

mapping_c1_c0 = create_patient_folds(common_patient_c1_c0, n_splits=5)

common_eeg_splits_c1_c01, common_labels_splits_c1_c01, common_patients_splits_c1_c01= split_data_by_patients_across_classes(
    common_data_c1_c01, common_labels_c1_c01, common_patients_c1_c01, mapping_c1_c0, n_splits=5)

common_eeg_splits_c1_c11, common_labels_splits_c1_c11, common_patients_splits_c1_c11 = split_data_by_patients_across_classes(
    common_data_c1_c11, common_labels_c1_c11, common_patients_c1_c11, mapping_c1_c0, n_splits=5)

for i in range(5):
    print(f"Fold {i+1}: EEG shape: {np.concatenate([common_eeg_splits_c1_c01[i], common_eeg_splits_c1_c11[i]]).shape}, "
          f"Labels shape: {np.concatenate([common_labels_splits_c1_c01[i], common_labels_splits_c1_c11[i]]).shape}, "
          f"Patients shape: {np.concatenate([common_patients_splits_c1_c01[i], common_patients_splits_c1_c11[i]]).shape}")

for i in range(5):
    print(f"Fold {i+1}: EEG shape 0: {common_eeg_splits_c1_c01[i].shape}, "
          f"Labels shape 0: {common_labels_splits_c1_c01[i].shape}, "
          f"Patients shape 0: {common_patients_splits_c1_c01[i].shape}")
print("__________________________________\n")

for i in range(5):
    print(f"Fold {i+1}: EEG shape 1: {common_eeg_splits_c1_c11[i].shape}, "
      f"Labels shape 1: {common_labels_splits_c1_c11[i].shape}, "
      f"Patients shape 1: {common_patients_splits_c1_c11[i].shape}")

print("__________________________________\n")

for i in range(5):
    print(f"No. of patients in class 1 is {len(np.unique(common_patients_splits_c1_c01[i]))}")
for i in range(5):
    print(f"No. of patients in class 0 is {len(np.unique(common_patients_splits_c1_c11[i]))}")
    
exclusive_patients_c1_c01 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0) - set(common_c2_c1)- set(only_c2) - set(common_patient_c1_c0))
exclusive_patients_c1_c11 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0))
exclusive_patients_c1_c21 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0))

print(len(exclusive_patients_c1_c01))
print(len(exclusive_patients_c1_c11))
print(len(exclusive_patients_c1_c21))



print("CLASS 1")

only_c1 = set(exclusive_patients_c1_c11)
only_c1= list(only_c1)
data_only_c1, labels_only_c1,patients_only_c1 = extract_common_data(patients_c1, data_c1, labels_c1, only_c1)
print(np.unique(patients_only_c1).shape)

c1_mapping = create_patient_folds(only_c1, n_splits=5)

eeg_splits_c1, labels_splits_c1, patients_splits_c1 = split_data_by_patients_across_classes(
    data_only_c1, labels_c1, patients_only_c1, c1_mapping, n_splits=5)

for i in range(5):
    print(f"No. of patients in class 1 is {len(np.unique(patients_splits_c1[i]))}")
    
exclusive_patients_all1_c01 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0) - set(common_c2_c1)- set(only_c2) - set(common_patient_c1_c0)- set(only_c1))
exclusive_patients_all1_c11 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0)- set(only_c1))
exclusive_patients_all1_c21 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0)- set(only_c1))

print(len(exclusive_patients_all1_c01))
print(len(exclusive_patients_all1_c11))
print(len(exclusive_patients_all1_c21))

print("CLASS 0")
only_c0 = set(exclusive_patients_all1_c01)
only_c0= list(only_c0)
data_only_c0, labels_only_c0,patients_only_c0 = extract_common_data(patients_c0, data_c0, labels_c0, only_c0)
print(np.unique(patients_only_c0).shape)

c0_mapping = create_patient_folds(only_c0, n_splits=5)

eeg_splits_c0, labels_splits_c0, patients_splits_c0 = split_data_by_patients_across_classes(
    data_only_c0, labels_c0, patients_only_c0, c0_mapping, n_splits=5)

for i in range(5):
    print(f"No. of patients in class 0 is {len(np.unique(patients_splits_c0[i]))}")

final_c0 = list(patients_set_c0 - set(common_patients_all) - set(common_c2_c0) - set(common_c2_c1)- set(only_c2) - set(common_patient_c1_c0)- set(only_c1) - set(only_c0))
final_c1 = list(patients_set_c1 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0)- set(only_c1)- set(only_c0))
final_c2 = list(patients_set_c2 - set(common_patients_all)- set(common_c2_c0) - set(common_c2_c1)- set(only_c2)- set(common_patient_c1_c0)- set(only_c1)- set(only_c0))

print(len(final_c0))
print(len(final_c1))
print(len(final_c2))

eeg_fold_1 = np.concatenate([eeg_012_c0[0], eeg_012_c1[0], eeg_012_c2[0], common_eeg_splits_c02[0], common_eeg_splits_c22[0], common_eeg_splits_c12[4], common_eeg_splits_1_c22[4],eeg_splits_c2[0],common_eeg_splits_c1_c01[0], common_eeg_splits_c1_c11[0],eeg_splits_c1[0],eeg_splits_c0[4]])
labels_fold_1 = np.concatenate([labels_012_c0[0], labels_012_c1[0], labels_012_c2[0],common_labels_splits_c02[0], common_labels_splits_c22[0],common_labels_splits_c12[4], common_labels_splits_1_c22[4],labels_splits_c2[0],common_labels_splits_c1_c01[0], common_labels_splits_c1_c11[0],labels_splits_c1[0],labels_splits_c0[4] ])
patients_fold_1 = np.concatenate([patients_012_c0[0], patients_012_c1[0], patients_012_c2[0],common_patients_splits_c02[0], common_patients_splits_c22[0],common_patients_splits_c12[4], common_patients_splits_1_c22[4],patients_splits_c2[0],common_patients_splits_c1_c01[0], common_patients_splits_c1_c11[0],patients_splits_c1[0],patients_splits_c0[4]])

eeg_fold_2 = np.concatenate([eeg_012_c0[1], eeg_012_c1[1], eeg_012_c2[1], common_eeg_splits_c02[1], common_eeg_splits_c22[1], common_eeg_splits_c12[3], common_eeg_splits_1_c22[3],eeg_splits_c2[1],common_eeg_splits_c1_c01[1], common_eeg_splits_c1_c11[1],eeg_splits_c1[1],eeg_splits_c0[3]])
labels_fold_2 = np.concatenate([labels_012_c0[1], labels_012_c1[1], labels_012_c2[1],common_labels_splits_c02[1], common_labels_splits_c22[1],common_labels_splits_c12[3], common_labels_splits_1_c22[3],labels_splits_c2[1],common_labels_splits_c1_c01[1], common_labels_splits_c1_c11[1],labels_splits_c1[1],labels_splits_c0[3] ])
patients_fold_2 = np.concatenate([patients_012_c0[1], patients_012_c1[1], patients_012_c2[1],common_patients_splits_c02[1], common_patients_splits_c22[1],common_patients_splits_c12[3], common_patients_splits_1_c22[3],patients_splits_c2[1],common_patients_splits_c1_c01[1], common_patients_splits_c1_c11[1],patients_splits_c1[1],patients_splits_c0[3]])

eeg_fold_3 = np.concatenate([eeg_012_c0[2], eeg_012_c1[2], eeg_012_c2[2], common_eeg_splits_c02[2], common_eeg_splits_c22[2], common_eeg_splits_c12[2], common_eeg_splits_1_c22[2],eeg_splits_c2[2],common_eeg_splits_c1_c01[2], common_eeg_splits_c1_c11[2],eeg_splits_c1[2],eeg_splits_c0[2]])
labels_fold_3 = np.concatenate([labels_012_c0[2], labels_012_c1[2], labels_012_c2[2],common_labels_splits_c02[2], common_labels_splits_c22[2],common_labels_splits_c12[2], common_labels_splits_1_c22[2],labels_splits_c2[2],common_labels_splits_c1_c01[2], common_labels_splits_c1_c11[2],labels_splits_c1[2],labels_splits_c0[2] ])
patients_fold_3 = np.concatenate([patients_012_c0[2], patients_012_c1[2], patients_012_c2[2],common_patients_splits_c02[2], common_patients_splits_c22[2],common_patients_splits_c12[2], common_patients_splits_1_c22[2],patients_splits_c2[2],common_patients_splits_c1_c01[2], common_patients_splits_c1_c11[2],patients_splits_c1[2],patients_splits_c0[2]])

eeg_fold_4 = np.concatenate([eeg_012_c0[3], eeg_012_c1[3], eeg_012_c2[3], common_eeg_splits_c02[3], common_eeg_splits_c22[3], common_eeg_splits_c12[1], common_eeg_splits_1_c22[1],eeg_splits_c2[3],common_eeg_splits_c1_c01[3], common_eeg_splits_c1_c11[3],eeg_splits_c1[3],eeg_splits_c0[1]])
labels_fold_4 = np.concatenate([labels_012_c0[3], labels_012_c1[3], labels_012_c2[3],common_labels_splits_c02[3], common_labels_splits_c22[3],common_labels_splits_c12[1], common_labels_splits_1_c22[1],labels_splits_c2[3],common_labels_splits_c1_c01[3], common_labels_splits_c1_c11[3],labels_splits_c1[3],labels_splits_c0[1] ])
patients_fold_4 = np.concatenate([patients_012_c0[3], patients_012_c1[3], patients_012_c2[3],common_patients_splits_c02[3], common_patients_splits_c22[3],common_patients_splits_c12[1], common_patients_splits_1_c22[1],patients_splits_c2[3],common_patients_splits_c1_c01[3], common_patients_splits_c1_c11[3],patients_splits_c1[3],patients_splits_c0[1]])

eeg_fold_5 = np.concatenate([eeg_012_c0[4], eeg_012_c1[4], eeg_012_c2[4], common_eeg_splits_c02[4], common_eeg_splits_c22[4], common_eeg_splits_c12[0], common_eeg_splits_1_c22[0],eeg_splits_c2[4],common_eeg_splits_c1_c01[4], common_eeg_splits_c1_c11[4],eeg_splits_c1[4],eeg_splits_c0[0]])
labels_fold_5 = np.concatenate([labels_012_c0[4], labels_012_c1[4], labels_012_c2[4],common_labels_splits_c02[4], common_labels_splits_c22[4],common_labels_splits_c12[0], common_labels_splits_1_c22[0],labels_splits_c2[4],common_labels_splits_c1_c01[4], common_labels_splits_c1_c11[4],labels_splits_c1[4],labels_splits_c0[0] ])
patients_fold_5 = np.concatenate([patients_012_c0[4], patients_012_c1[4], patients_012_c2[4],common_patients_splits_c02[4], common_patients_splits_c22[4],common_patients_splits_c12[0], common_patients_splits_1_c22[0],patients_splits_c2[4],common_patients_splits_c1_c01[4], common_patients_splits_c1_c11[4],patients_splits_c1[4],patients_splits_c0[0]])

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

DATA BALANCER AND EARLY STOPPING


In [ ]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




results plots


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
    auc,
    f1_score,
    balanced_accuracy_score,
    classification_report,
)
from sklearn.preprocessing import label_binarize

def plot_metrics(labels, predictions, n_classes=3):
    """
    Plots and computes metrics for classification tasks.

    Args:
        labels (array-like): True labels.
        predictions (array-like): Predicted probabilities or class predictions.
        n_classes (int): Number of classes (default is 3 for multi-class classification).
    """
    # If predictions are probabilities, convert to class predictions
    if predictions.ndim > 1:
        predicted_classes = np.argmax(predictions, axis=1)
    else:
        predicted_classes = predictions

    # Confusion Matrix
    cm = confusion_matrix(labels, predicted_classes)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

    # Class-wise Accuracy
    print("\nClass-wise Accuracy:")
    for d in range(n_classes):
        correct_preds = cm[d][d]
        total_true_samples = sum(cm[d])
        ratio_correct = correct_preds / total_true_samples if total_true_samples != 0 else 0
        print(f'Class {d}: Correct Predictions / Total True Samples = {correct_preds}/{total_true_samples} ({ratio_correct:.2%})')

    # Precision-Recall Curves and AUPRC
    print("\nPrecision-Recall Curves:")
    labels_binarized = label_binarize(labels, classes=np.arange(n_classes))
    auprcs = []
    for class_idx in range(n_classes):
        precision, recall, _ = precision_recall_curve(labels_binarized[:, class_idx], predictions[:, class_idx])
        auprc = auc(recall, precision)
        auprcs.append(auprc)
        plt.plot(recall, precision, label=f'Class {class_idx + 1} (AUPRC = {auprc:.2f})')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curves for each class')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

    # ROC Curves and AUC
    print("\nROC Curves:")
    for class_idx in range(n_classes):
        fpr, tpr, _ = roc_curve(labels_binarized[:, class_idx], predictions[:, class_idx])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'Class {class_idx + 1} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for each class')
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

    # Mean F1 Score
    f1 = f1_score(labels, predicted_classes, average='macro')
    print(f"\nMean F1 Score: {f1:.4f}")

    # Balanced Accuracy
    balanced_acc = balanced_accuracy_score(labels, predicted_classes)
    print(f"Balanced Accuracy: {balanced_acc:.4f}")

    # Average AUPRC
    mean_auprc = np.mean(auprcs)
    print(f"Average AUPRC: {mean_auprc:.4f}")

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(labels, predicted_classes))

    return {
        "confusion_matrix": cm,
        "class_wise_accuracy": [cm[d][d] / sum(cm[d]) if sum(cm[d]) != 0 else 0 for d in range(n_classes)],
        "mean_f1_score": f1,
        "balanced_accuracy": balanced_acc,
        "average_auprc": mean_auprc,
        "auprc_per_class": auprcs,
    }

model and testing with random data


In [46]:
import torch
# Debug mode flag
DEBUG_MODE = True

import torch.nn as nn
import torch.nn.functional as F

class EEGNet(nn.Module):
    def __init__(self, num_classes = 3,num_channels = 20 , num_timepoints = 5120):
        super(EEGNet, self).__init__()
        self.T = num_timepoints
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, num_channels), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        self.fc1 = nn.Linear(2560, num_classes)
        

    def forward(self, x):
        if DEBUG_MODE: print(f"Input shape: {x.shape}")
        
        # Layer 1
        x = F.elu(self.conv1(x))
        if DEBUG_MODE: print(f"After conv1: {x.shape}")
        x = self.batchnorm1(x)
        if DEBUG_MODE: print(f"After batchnorm1: {x.shape}")
        x = F.dropout(x, 0.25)
        if DEBUG_MODE: print(f"After dropout1: {x.shape}")
        x = x.permute(0, 3, 1, 2)
        if DEBUG_MODE: print(f"After permute: {x.shape}")
        
        # Layer 2
        x = self.padding1(x)
        if DEBUG_MODE: print(f"After padding1: {x.shape}")
        x = F.elu(self.conv2(x))
        if DEBUG_MODE: print(f"After conv2: {x.shape}")
        x = self.batchnorm2(x)
        if DEBUG_MODE: print(f"After batchnorm2: {x.shape}")
        x = F.dropout(x, 0.25)
        if DEBUG_MODE: print(f"After dropout2: {x.shape}")
        x = self.pooling2(x)
        if DEBUG_MODE: print(f"After pooling2: {x.shape}")
        
        # Layer 3
        x = self.padding2(x)
        if DEBUG_MODE: print(f"After padding2: {x.shape}")
        x = F.elu(self.conv3(x))
        if DEBUG_MODE: print(f"After conv3: {x.shape}")
        x = self.batchnorm3(x)
        if DEBUG_MODE: print(f"After batchnorm3: {x.shape}")
        x = F.dropout(x, 0.25)
        if DEBUG_MODE: print(f"After dropout3: {x.shape}")
        x = self.pooling3(x)
        if DEBUG_MODE: print(f"After pooling3: {x.shape}")
        
        # FC Layer
        x = x.reshape(-1, 4*2*x.size(3))
        if DEBUG_MODE: print(f"After flattening: {x.shape}")
        x = F.sigmoid(self.fc1(x))
        
        return x

In [47]:
model = EEGNet()


demo_input = torch.randn(32, 1, 5120,20)

# Run forward pass
output = model(demo_input)

print("Output shape:", output.shape)
print("Output:", output)


Input shape: torch.Size([32, 1, 5120, 20])
After conv1: torch.Size([32, 16, 5120, 1])
After batchnorm1: torch.Size([32, 16, 5120, 1])
After dropout1: torch.Size([32, 16, 5120, 1])
After permute: torch.Size([32, 1, 16, 5120])
After padding1: torch.Size([32, 1, 17, 5153])
After conv2: torch.Size([32, 4, 16, 5122])
After batchnorm2: torch.Size([32, 4, 16, 5122])
After dropout2: torch.Size([32, 4, 16, 5122])
After pooling2: torch.Size([32, 4, 4, 1281])
After padding2: torch.Size([32, 4, 11, 1284])
After conv3: torch.Size([32, 4, 4, 1281])
After batchnorm3: torch.Size([32, 4, 4, 1281])
After dropout3: torch.Size([32, 4, 4, 1281])
After pooling3: torch.Size([32, 4, 2, 320])
After flattening: torch.Size([32, 2560])
Output shape: torch.Size([32, 3])
Output: tensor([[0.8952, 0.1927, 0.2391],
        [0.9151, 0.5152, 0.2031],
        [0.9263, 0.2506, 0.3068],
        [0.8935, 0.3210, 0.2142],
        [0.8907, 0.1687, 0.3080],
        [0.9059, 0.4850, 0.4131],
        [0.9130, 0.1579, 0.1975],
  

In [48]:
DEBUG_MODE = False

In [49]:
from torchinfo import summary
# Print the model summary
summary(model, input_size=(32, 1, 5120, 20), col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"], row_settings=["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
EEGNet (EEGNet)                          [32, 1, 5120, 20]         [32, 3]                   --                        --                        --
├─Conv2d (conv1)                         [32, 1, 5120, 20]         [32, 16, 5120, 1]         336                       [1, 20]                   55,050,240
├─BatchNorm2d (batchnorm1)               [32, 16, 5120, 1]         [32, 16, 5120, 1]         32                        --                        1,024
├─ZeroPad2d (padding1)                   [32, 1, 16, 5120]         [32, 1, 17, 5153]         --                        --                        --
├─Conv2d (conv2)                         [32, 1, 17, 5153]         [32, 4, 16, 5122]         260                       [2, 32]                   681,840,640
├─BatchNorm2d (batchnorm2)               [32, 4, 16, 5122]         [32, 4, 16, 5122] 

training code

In [ ]:
import pandas as pd
import os
import datetime


# Fold indices and other configurations
num_folds = 5
fold_indices = np.random.permutation(np.arange(num_folds))
val_fold_indices = np.roll(fold_indices, 1)

test_folds_chosen = []
val_folds_chosen = []
fold_confusion_matrices = []  # To store confusion matrices for each fold
fold_accuracies = []  # To store balanced accuracy for each fold
fold_auprcs = []  # To store AUPRC for each fold


learning_rate = 0.001
batch_size = 32

num_classes = 3
n_classes = 3
epochs = 100

device = 'cuda' if torch.cuda.is_available() else 'cpu'


def save_results_to_csv(fold_accuracies, fold_auprcs, fold_confusion_matrices, model_info, csv_path=r"F:\CODING\py\newnotebooks\results.csv"):
    """
    Save all results from the current experiment to a CSV file

    Parameters:
    - fold_accuracies: list of balanced accuracy scores for each fold
    - fold_auprcs: list of AUPRC scores for each fold
    - fold_confusion_matrices: list of confusion matrices for each fold
    - model_info: string with model architecture description
    - csv_path: path to the CSV file to save results
    """
    # Current time for experiment identification
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Prepare data for the DataFrame
    data = {
        "timestamp": timestamp,
        "model_info": model_info,
        "num_classes": num_classes,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "mean_balanced_accuracy": np.mean(fold_accuracies),
        "std_balanced_accuracy": np.std(fold_accuracies),
        "mean_auprc": np.mean(fold_auprcs),
        "std_auprc": np.std(fold_auprcs),
    }

    # Add individual fold results
    for i, (acc, auprc) in enumerate(zip(fold_accuracies, fold_auprcs)):
        data[f"fold_{i+1}_accuracy"] = acc
        data[f"fold_{i+1}_auprc"] = auprc

    # Add confusion matrix info
    for i, cm in enumerate(fold_confusion_matrices):
        data[f"fold_{i+1}_confusion_matrix"] = str(cm)

    # Create DataFrame and append to CSV
    df = pd.DataFrame([data])

    # Check if file exists
    file_exists = os.path.isfile(csv_path)

    # Save to CSV
    if file_exists:
        df.to_csv(csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(csv_path, mode='w', header=True, index=False)

    print(f"Results saved to {csv_path}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, auc




# Training loop for cross-validation
for fold_idx in range(num_folds):
    print(f'Fold No: {fold_idx + 1}')

    # Initialize model, loss, and optimizer
    fold_model = EEGNet(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.3)
    optimizer = optim.Adam(fold_model.parameters(), lr=learning_rate)

    # Split data into train, validation, and test sets
    test_fold = fold_indices[fold_idx]
    val_fold = val_fold_indices[fold_idx]
    train_folds = [fold for fold in fold_indices if fold != test_fold and fold != val_fold]

    train_data = np.concatenate([eeg_folds[j] for j in train_folds])
    train_labels = np.concatenate([labels_folds[j] for j in train_folds])



    test_folds_chosen.append(test_fold)
    val_folds_chosen.append(val_fold)

    early_stopping = EarlyStopping(patience=10)

    # Training loop
    for epoch in range(epochs):
        balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=1)

        train_dataset = TensorDataset(
            torch.tensor(balanced_train_data, dtype=torch.float32).to(device),
            torch.tensor(balanced_train_labels, dtype=torch.long).to(device)
        )
        train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

        fold_model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = fold_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = balanced_accuracy_score(all_labels, all_preds)
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

        # Validation loop
        val_data = eeg_folds[val_fold]
        val_data = val_data
        val_labels = labels_folds[val_fold]
        val_dataset = TensorDataset(
            torch.tensor(val_data, dtype=torch.float32).to(device),
            torch.tensor(val_labels, dtype=torch.long).to(device)
        )
        val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

        fold_model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels_list = []

        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_outputs = fold_model(val_inputs)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()

                _, val_batch_preds = torch.max(val_outputs, 1)
                val_preds.extend(val_batch_preds.cpu().numpy())
                val_labels_list.extend(val_labels.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = balanced_accuracy_score(val_labels_list, val_preds)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}')

        early_stopping(val_loss, fold_model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

    # Load the best model
    early_stopping.load_best_model(fold_model)

    # Test loop
    test_data = eeg_folds[test_fold]
    test_data = test_data
    test_labels = labels_folds[test_fold]
    test_dataset = TensorDataset(
        torch.tensor(test_data, dtype=torch.float32).to(device),
        torch.tensor(test_labels, dtype=torch.long).to(device)
    )
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    fold_model.eval()
    test_probs = []
    test_preds = []
    test_labels_list = []

    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_outputs = fold_model(test_inputs)
            probabilities = torch.softmax(test_outputs, dim=1)
            test_probs.extend(probabilities.cpu().numpy())
            _, preds = torch.max(probabilities, 1)
            test_preds.extend(preds.cpu().numpy())
            test_labels_list.extend(test_labels.cpu().numpy())

    # Compute metrics
    test_acc = balanced_accuracy_score(test_labels_list, test_preds)
    fold_accuracies.append(test_acc)

    cm = confusion_matrix(test_labels_list, test_preds)
    fold_confusion_matrices.append(cm)

    test_labels_binarized = label_binarize(test_labels_list, classes=np.arange(num_classes))
    test_auprcs = []
    for class_idx in range(num_classes):
        precision, recall, _ = precision_recall_curve(test_labels_binarized[:, class_idx], np.array(test_probs)[:, class_idx])
        auprc = auc(recall, precision)
        test_auprcs.append(auprc)

    mean_test_auprc = np.mean(test_auprcs)
    fold_auprcs.append(mean_test_auprc)

    print(f'Test Fold {fold_idx + 1}, Mean AUPRC: {mean_test_auprc:.4f}, Balanced Accuracy: {test_acc:.4f}')

    # Use the plot_metrics function to visualize metrics
    plot_metrics(np.array(test_labels_list), np.array(test_probs), n_classes=num_classes)

# Final metrics across all folds
average_auprc = np.mean(fold_auprcs)
mean_accuracy = np.mean(fold_accuracies)
print(f'Accuracy for each fold: {fold_accuracies}')
print(f'AUPRC for each fold: {fold_auprcs}')
print(f'Average AUPRC across all folds: {average_auprc}')
print(f'Average Balanced Accuracy across all folds: {mean_accuracy}')



In [ ]:
#save results to results.csv
save_path = r"SAVE PATH HERE"
spect = "MEL"
save_results_to_csv(fold_accuracies,fold_auprcs,fold_confusion_matrices,model_info=f"EEGNET MODEL {num_classes}classes {spect} spectrogram",csv_path=save_path)